### Standardization of SUPERCIAS's data.

The objective of this notebook is to automate the preprocessing and generation of the financial statements of all the companies listed in the Superintendencia de Compañías, Valores y Seguros. The information is quite complete, but the aggregated accounts tend to be imperfectly registered, which is why it's necessary to fill them automatically using the values of the accounts that compose them.

This task can be done using the IFRS manual developed by SUPERCIAS; which details with accuracy which accounts compose the faulty aggregated variables before mentioned. The general idea is to sum up those smaller accounts to get a better version of the faulty variables.

In [2]:
#Import Libraries
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
import tabula

Before working on the standardization it is crucial to find out how the aggregation method of the Ecuadorian Superintendency of Companies, Securities, and Insurance works. To do this we decided to grab the largest company, in terms of revenue (i.e. Corporación Favorita C.A.). The reasoning behind this is that such a large company ought to have the least empty values possible for the disaggregated accounts; therefore making it easier to make a complete standardization of the methods needed to generate the aggregated accounts.

We will read the last balance sheet and income statement submitted by this company, then we will use the IFRS instructive developed by the Superintendency to design the aggregation methodology. The idea is to create a function that can take in the value of a specific account, and sum up all the disaggregated "children" of that account.

In [5]:
#Read the submitted statements from the pdf.
fav = tabula.read_pdf("Guide_Docs/bs_favorita_2022.pdf",pages = "all")
for i in range(len(fav)):
    if i == 0:
        num = 7
    else:
        num = 0
    cols = fav[i].iloc[num]
    fav[i].columns = cols
    fav[i] = fav[i].iloc[num+1:].reset_index(drop = True)
fav = fav[:-1]
fav = pd.concat(fav)
#Set dtypes for each column
fav["CUENTA"] = fav["CUENTA"].apply(str)
fav["CÓDIGO"] = fav["CÓDIGO"].apply(str)
fav["VALOR (En USD$)"] = fav["VALOR (En USD$)"].astype(float)
#Visualize data as units
pd.options.display.float_format = '{:.2f}'.format
fav


,CUENTA,CÓDIGO,VALOR (En USD$)
0,ACTIVO,1,2480403867.05
1,ACTIVO CORRIENTE,101,701635273.03
2,EFECTIVO Y EQUIVALENTES DE EFECTIVO,10101,7283311.87
3,CAJA,1010101,1342366.55
4,INSTITUCIONES FINANCIERAS PÚBLICAS,1010102,0.00
...,...,...,...
48,SUPERÁVIT POR REVALUACIÓN DE INVERSIONES,30607,0.00
49,RESULTADOS DEL EJERCICIO,307,152679114.47
50,GANANCIA NETA DEL PERIODO,30701,152679114.47
51,(-) PÉRDIDA NETA DEL PERIODO,30702,0.00


In [6]:
set(fav["CÓDIGO"].str.len())

{1, 2, 3, 5, 7, 9, 11}

In [7]:
#Display parent
print("Displaying Total Assets account:")
display(fav[fav["CÓDIGO"] == "1"])
#Display children
print("What children is the Total Assets account composed of?")
display(fav[(fav["CÓDIGO"].str.startswith("1")) & (fav["CÓDIGO"].str.len() == 3)])
s = fav[(fav["CÓDIGO"].str.startswith("1")) & (fav["CÓDIGO"].str.len() == 3)]["VALOR (En USD$)"].sum()
print(f"Total sum of children = {s}")
#Test if parent matches the sum of children
print(f"""Is sum of children equal to parent?: {fav[fav["CÓDIGO"] == "1"]["VALOR (En USD$)"][0] == s}""")

Displaying Total Assets account:


,CUENTA,CÓDIGO,VALOR (En USD$)
0,ACTIVO,1,2480403867.05


What children is the Total Assets account composed of?


,CUENTA,CÓDIGO,VALOR (En USD$)
1,ACTIVO CORRIENTE,101,701635273.03
3,ACTIVOS NO CORRIENTES,102,1778768594.02


Total sum of children = 2480403867.05
Is sum of children equal to parent?: True


According to the syntax of the account numeration, as well as the IFRS instructive, the accounts follow a specific logic. All accounts have an odd number of digits. Their children contain the same characters as their parents, plus two other characters more. For example, the Total Assets account (1) is composed of the Current Assets (101) and the Non Current Assets (102) accounts. And the same goes for the accounts that compose the current assets and non current assets accounts which are, respectively, (101XX) & (102XX).

Now while this logic works well with some accounts it might fail for those accounts that take away from the total, but are registered with positive values. It is necessary to identify these accounts, preferably in an automated fashion. Then, it will be useful to apply absolute value to each of their values and turn them all negative again; just in case there was an error uploading the numbers.

In [8]:
display(fav[fav["CUENTA"].str.contains("\(-\)")])
#Negative value function
def negate(df):
    p = -abs(df[df["CUENTA"].str.contains(r"\(-\)")]["VALOR (En USD$)"])
    df.loc[df["CUENTA"].str.contains(r"\(-\)"),"VALOR (En USD$)"] = p
negate(fav)
fav[fav["CUENTA"].str.contains("\(-\)")]

,CUENTA,CÓDIGO,VALOR (En USD$)
45,(-) PROVISIÓN POR VALOR NETO DE REALIZACIÓN Y ...,1010313,0.00
16,"(-) DEPRECIACIÓN ACUMULADA PROPIEDADES, PLANTA...",1020112,-476943778.97
17,"(-) DETERIORO ACUMULADO DE PROPIEDADES, PLANT...",1020113,0.00
20,(-) AMORTIZACION ACUMULADA DE ACTIVOS DE EXPLO...,102011402,0.00
21,(-) DETERIORO ACUMULADO DE ACTIVOS DE EXPLORA...,102011403,0.00
29,(-) DEPRECIACION ACUMULADA DE PROPIEDADES DE I...,1020203,-35723314.36
30,(-) DETERIORO ACUMULADO DE PROPIEDADES DE INVE...,1020204,0.00
36,(-) DEPRECIACION ACUMULADA DE ACTIVOS BIOLÓGICOS,1020305,0.00
37,(-) DETERIORO ACUMULADO DE ACTIVOS BIOLOGÍCOS,1020306,0.00
43,(-) AMORTIZACIÓN ACUMULADA DE ACTIVOS INTANGIBLE,1020405,-36427660.12


,CUENTA,CÓDIGO,VALOR (En USD$)
45,(-) PROVISIÓN POR VALOR NETO DE REALIZACIÓN Y ...,1010313,-0.00
16,"(-) DEPRECIACIÓN ACUMULADA PROPIEDADES, PLANTA...",1020112,-476943778.97
17,"(-) DETERIORO ACUMULADO DE PROPIEDADES, PLANT...",1020113,-0.00
20,(-) AMORTIZACION ACUMULADA DE ACTIVOS DE EXPLO...,102011402,-0.00
21,(-) DETERIORO ACUMULADO DE ACTIVOS DE EXPLORA...,102011403,-0.00
29,(-) DEPRECIACION ACUMULADA DE PROPIEDADES DE I...,1020203,-35723314.36
30,(-) DETERIORO ACUMULADO DE PROPIEDADES DE INVE...,1020204,-0.00
36,(-) DEPRECIACION ACUMULADA DE ACTIVOS BIOLÓGICOS,1020305,-0.00
37,(-) DETERIORO ACUMULADO DE ACTIVOS BIOLOGÍCOS,1020306,-0.00
43,(-) AMORTIZACIÓN ACUMULADA DE ACTIVOS INTANGIBLE,1020405,-36427660.12


#### Designing the aggregation function

In [9]:
#Function that checks if the sum of immediate children is equal to the value of the aggregate account
def checkSum(df,acct):
    df1 = df.copy(deep = True)
    acct = str(acct)
    #Children dframe
    a = df[(df["CÓDIGO"].str.startswith(acct)) & (df["CÓDIGO"].str.len() == len(acct)+2)]
    #Keep track of the number of children available for the account
    print(f"Size of children dframe: {len(a)}")

    o_value = round(df[(df["CÓDIGO"].str.startswith(acct)) & (df["CÓDIGO"].str.len() == len(acct))]["VALOR (En USD$)"].values[0],2)
    n_value = round(a["VALOR (En USD$)"].sum(),2)
    print(f"nvalue {n_value}")
    print("Digits in children")
    display(len(a["CÓDIGO"].tolist()[0]))
    print("Children df:")
    display(a)
    print(f"""Sum of children = {n_value}""")
    print(f"""Actual value of Account: {o_value}""")
    if o_value == n_value:
        print("Sum of children = Aggregated Account")
    else:
        print("Sum of children != Aggregated Account")
    df1.loc[df1["CÓDIGO"] == acct, "VALOR (En USD$)"] = n_value
    stuff = df1[(df1["CÓDIGO"].str.startswith(acct)) & (df1["CÓDIGO"].str.len() == len(acct))]["VALOR (En USD$)"].values[0]
    display(round(stuff,2))
checkSum(fav,10201)

Size of children dframe: 14
nvalue 842886295.83
Digits in children


7

Children df:


,CUENTA,CÓDIGO,VALOR (En USD$)
5,TERRENOS,1020101,198331581.21
6,EDIFICIOS,1020102,364717482.41
7,CONSTRUCCIONES EN CURSO,1020103,35391192.48
8,INSTALACIONES,1020104,0.00
9,MUEBLES Y ENSERES,1020105,25416497.26
10,MAQUINARIA Y EQUIPO,1020106,313521038.97
11,"NAVES, AEREONAVES, BARCAZAS Y SIMILARES",1020107,0.00
12,EQUIPO DE COMPUTACIÓN,1020108,61823261.94
13,"VEHÍCULOS, EQUIPOS DE TRASPORTE Y EQUIPO CAMIN...",1020109,19212204.50
14,"OTROS PROPIEDADES, PLANTA Y EQUIPO",1020110,301416816.03


Sum of children = 842886295.83
Actual value of Account: 842886295.83
Sum of children = Aggregated Account


842886295.83

In [10]:
#Function that recursively sums up the childrens of accounts to replace them with the value of the aggregated account

"""It identifies children for an account, and then it identifies the children of the children, recursively, 
until there are no more children, at which point it sums up all of the children to replace this sum with the value of 
the aggregate account"""

def sumup(df,acct):
    acct = str(acct)
    #Children dframe
    children = df[(df["CÓDIGO"].str.startswith(acct)) & (df["CÓDIGO"].str.len() == len(acct)+2)]
    #Check for presence of children
    if children.empty:
        return df[df["CÓDIGO"] == acct]["VALOR (En USD$)"].values[0]
    else:
        sum_children = 0
        for index,child in children.iterrows():
            sum_children += sumup(df,child["CÓDIGO"])

        #If it's the equity account, sum account 31, which was registered in pair numbers of digits for an unknown reason
        if acct == "3":
            sum_children += df[df["CÓDIGO"] == "31"]["VALOR (En USD$)"].values[0]
        #Update the parent's value with the sum of children
        df.loc[df["CÓDIGO"] == acct,"VALOR (En USD$)"] = sum_children
        return round(sum_children,2)
sumup(fav,10201)

842886295.83

With this recursive implementation we ensure that every parent from the balance sheet is calculated based on the sum of their children. This can be used on the massive data from the balances to achieve a more complete information for those companies that have values of zero for their aggregate accounts, but non-zero values for the children accounts.

In [15]:
#Decompress 2022's financial statements
with ZipFile("Financials/estadosFinancieros_2022.zip","r") as zip_ref:
    zip_ref.extractall("Financials")
#Read Financial Statements ¿
df = pd.read_csv("Financials/balances_2022_1.txt",sep = "\t",encoding = "latin-1")
dic = pd.read_csv("Financials/catalogo_2022_1.txt",sep = "\t",encoding = "latin-1",header = None, names = ["CÓDIGO","CUENTA"])
display(df.head(5))
display(dic.head(5))
# Remove files once read by pandas
for i in [i for i in os.listdir("Financials") if "balances" in i or "catalogo" in i]:
    os.remove(f"Financials/{i}")
#Set the correct dtypes
for i in df.columns:
    df[i] = df[i].astype(str)
#Set numeric variables to string.
for i in [i for i in df.columns if "CUENTA" in i]:
    df[i] = df[i].apply(lambda x : x.replace(",","."))
    df[i] = df[i].apply(float)
display(df.dtypes)

C:\Users\edydu\AppData\Local\Temp\ipykernel_18836\4253750676.py:5: DtypeWarning: Columns (15,17,18,19,20,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,50,51,52,53,54,55,56,57,58,59,60,61,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,92,93,94,95,96,97,98,99,101,102,103,104,105,106,107,108,109,110,111,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,151,152,155,164,165,166,176,180,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,204,207,209,212,213,214,215,216,217,218,219,220,221,222,224,225,226,227,228,230,231,232,236,237,239,240,241,242,245,246,253,254,258,261,262,263,264,265,266,267,274,283,284,285,287,290,291,293,296,297,298,299,300,301,302,303,304,305,306,308,310,313,316,317,318,319,322,326,327,329,330,331,332,333,334,335,336,337,338,339,340,341,342,346,347,348,349,356,357,358,359,360,362,365,366,367,368,369,370,373,374,375,376,377,378,382,387,388,391,392,394,397,398,

,AÑO,EXPEDIENTE,RUC,NOMBRE,RAMA_ACTIVIDAD,DESCRIPCION_RAMA,CIIU,CUENTA_1,CUENTA_101,CUENTA_10101,...,CUENTA_80004,CUENTA_80005,CUENTA_80006,CUENTA_80007,CUENTA_80008,CUENTA_80009,CUENTA_801,CUENTA_80101,CUENTA_80102,Unnamed: 629
0,2022,1,1790013731001,ACEITES TROPICALES SOCIEDAD ANONIMA ATSA,A,"AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA.",A0126.01,"1360051,92","71630,18","54744,04",...,0,"-1244813,70",0,0,0,0,"-1288545,54",0,0,NaN
1,2022,2,1790004724001,ACERIA DEL ECUADOR CA ADELCA.,C,INDUSTRIAS MANUFACTURERAS.,C2410.25,"459045627,92","241767990,46","41809089,24",...,0,0,0,0,0,0,0,0,0,NaN
2,2022,3,1790008959001,ACERO COMERCIAL ECUATORIANO S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4610.03,"11224470,90","10893439,61","1238466,08",...,0,0,0,0,0,0,0,0,0,NaN
3,2022,11,1790044149001,AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,H,TRANSPORTE Y ALMACENAMIENTO.,H5110.01,"20915428,96","2181954,71","2050454,71",...,0,0,0,0,0,0,0,0,0,NaN
4,2022,22,1790023516001,AGENCIAS Y REPRESENTACIONES CORDOVEZ SA,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4630.95,"27799662,03","27131380,13","1212276,74",...,0,"1402,76",0,0,0,0,"1010213,23",0,0,NaN


,CÓDIGO,CUENTA
0,1,ACTIVO
1,101,ACTIVO CORRIENTE
2,10101,EFECTIVO Y EQUIVALENTES DE EFECTIVO
3,1010101,CAJA
4,1010102,INSTITUCIONES FINANCIERAS PÚBLICAS


AÑO                object
EXPEDIENTE         object
RUC                object
NOMBRE             object
RAMA_ACTIVIDAD     object
                   ...   
CUENTA_80009      float64
CUENTA_801        float64
CUENTA_80101      float64
CUENTA_80102      float64
Unnamed: 629       object
Length: 630, dtype: object

To illustrate the aggregation problem that motivates this stage of the project, we can filter for companies that have zero values in the Total Assets account (CUENTA_1), but non-zero values for the immediate children of Current Assets account (CUENTA_101).

In [16]:
#Illustrate normal behavior of accounts
a = df[(df["CUENTA_1"] != 0) & (df["CUENTA_101"] != 0)][["NOMBRE","CUENTA_1","CUENTA_101","CUENTA_102"]]
a["SUM_101+102"] = a["CUENTA_101"]+a["CUENTA_102"]
print("Examples of companies with correctly registered accounts:")
display(a.head(5))
print(f"""Is the sum of immediate children equal to aggregate account?: {a.iloc[0]["CUENTA_1"] == a.iloc[0]["SUM_101+102"]}\n\n""")

#Illustrate anomalous behavior of accounts
a = df[(df["CUENTA_1"] == 0) & (df["CUENTA_101"] != 0)][["NOMBRE","CUENTA_1","CUENTA_101","CUENTA_102"]]
a["SUM_101+102"] = a["CUENTA_101"]+a["CUENTA_102"]
print("Examples of companies which aggregate accounts are not the sum of immediate children:")
display(a.head(5).sort_values("CUENTA_101",ascending=False))
print(f"""Is the sum of immediate children equal to aggregate account?: {a.iloc[0]["CUENTA_1"] == a.iloc[0]["SUM_101+102"]}\n\n""")
#Save the index of anomalous companies for testing the aggregation function.
ind = a.head(5).index

Examples of companies with correctly registered accounts:


,NOMBRE,CUENTA_1,CUENTA_101,CUENTA_102,SUM_101+102
0,ACEITES TROPICALES SOCIEDAD ANONIMA ATSA,1360051.92,71630.18,1288421.74,1360051.92
1,ACERIA DEL ECUADOR CA ADELCA.,459045627.92,241767990.46,217277637.46,459045627.92
2,ACERO COMERCIAL ECUATORIANO S.A.,11224470.90,10893439.61,331031.29,11224470.90
3,AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,20915428.96,2181954.71,18733474.25,20915428.96
4,AGENCIAS Y REPRESENTACIONES CORDOVEZ SA,27799662.03,27131380.13,668281.90,27799662.03


Is the sum of immediate children equal to aggregate account?: True


Examples of companies which aggregate accounts are not the sum of immediate children:


,NOMBRE,CUENTA_1,CUENTA_101,CUENTA_102,SUM_101+102
91855,RESTAURANTES MULTIFOOD-ECUADOR S.A.S.,0.00,98980.88,0.00,98980.88
60152,IMPORTADORA GOMEZ&SANCHEZ S.A.S.,0.00,300.00,0.00,300.00
61969,VIFCAR S.A.S.,0.00,300.00,0.00,300.00
60255,ELSALLER S.A.S.,0.00,100.00,0.00,100.00
37254,ARICASEG CIA. LTDA.,0.00,0.01,0.00,0.01


Is the sum of immediate children equal to aggregate account?: False




The issue with the anomalous accounts presented above is that the information provided is less consistent and more inaccurate. This problem can be addressed by utilizing the 'sumup' method previously defined, enabling us to obtain a more reliable set of information to work with.

However, the 'sumup' method defined earlier is designed to work with a single column where accounts are stored in rows. In this dataset, accounts are represented as individual columns. To make the function compatible with this extensive dataset, we'll need to implement some modifications.

In [21]:
def sumup_bs(df,acct):
    acct = f"CUENTA_{acct}"
    cols = df.columns
    #List of Immediate Children
    children = [i for i in cols if (i.startswith(acct)) & (len(i) == len(acct)+2)]

    #Check for presence of children
    if not bool(children):
        return df[acct]
    else:
        #Aggregate children for all companies
        sum_children = np.zeros(len(df))
        for child in children:
            sum_children += sumup_bs(df,child.replace("CUENTA_",""))

        #If it's the equity account, sum account 31, which was registered in pair numbers of digits for an unknown reason
        if acct == "3":
            sum_children += df["CUENTA_31"]
        #Update the parent's value with the sum of children
        df[acct] = sum_children
        return sum_children
#Make a deep copy of the original dataframe so as to compare the changes the function applies.
df1 = df.copy(deep = True)
sumup_bs(df1,1);

In [24]:
def test_sum(df,df1):
    #Testing the results of the aggregation function
    #Previous behavior of anomalous companies
    a = df[(df["CUENTA_1"] == 0) & (df["CUENTA_101"] != 0)][["NOMBRE","CUENTA_1","CUENTA_101","CUENTA_102"]]
    a["SUM_101+102"] = a["CUENTA_101"]+a["CUENTA_102"]
    print("Previous Behavior of anomalous accounts:")
    display(a.head(5).sort_values("CUENTA_101",ascending=False))
    print(f"""Is the sum of immediate children equal to aggregate account?: {a.iloc[0]["CUENTA_1"] == a.iloc[0]["SUM_101+102"]}\n\n""")
    #Save the index of anomalous companies for testing the aggregation function.
    ind = a.head(5).index

    #Illustrate anomalous behavior of accounts
    a = df1.iloc[ind][["NOMBRE","CUENTA_1","CUENTA_101","CUENTA_102"]]
    a["SUM_101+102"] = a["CUENTA_101"]+a["CUENTA_102"]
    print("New behavior of anomalous accounts:")
    display(a.head(5).sort_values("CUENTA_101",ascending=False))
    print(f"""Is the sum of immediate children equal to aggregate account?: {a.iloc[0]["CUENTA_1"] == a.iloc[0]["SUM_101+102"]}\n\n""")
test_sum(df,df1)

Previous Behavior of anomalous accounts:


,NOMBRE,CUENTA_1,CUENTA_101,CUENTA_102,SUM_101+102
91855,RESTAURANTES MULTIFOOD-ECUADOR S.A.S.,0.00,98980.88,0.00,98980.88
60152,IMPORTADORA GOMEZ&SANCHEZ S.A.S.,0.00,300.00,0.00,300.00
61969,VIFCAR S.A.S.,0.00,300.00,0.00,300.00
60255,ELSALLER S.A.S.,0.00,100.00,0.00,100.00
37254,ARICASEG CIA. LTDA.,0.00,0.01,0.00,0.01


Is the sum of immediate children equal to aggregate account?: False


New behavior of anomalous accounts:


,NOMBRE,CUENTA_1,CUENTA_101,CUENTA_102,SUM_101+102
91855,RESTAURANTES MULTIFOOD-ECUADOR S.A.S.,125030.26,98980.88,26049.38,125030.26
60152,IMPORTADORA GOMEZ&SANCHEZ S.A.S.,300.00,300.00,0.00,300.00
61969,VIFCAR S.A.S.,300.00,300.00,0.00,300.00
37254,ARICASEG CIA. LTDA.,0.00,0.00,0.00,0.00
60255,ELSALLER S.A.S.,0.00,0.00,0.00,0.00


Is the sum of immediate children equal to aggregate account?: True




This demonstrates that the aggregation problem can be promptly addressed using the logic from the 'sumup' function. It also enables us to identify and aggregate accounts that initially appeared to have no issues within the accounts' tree structure. In the provided examples, the function successfully identified that the Non-Current Assets account (102) was also inaccurately aggregated. It proceeded to calculate the sum of all its children (and their descendants) to derive a more precise total.

While this method works perfectly for year 2022's financial information, there are a few caveats that should be noted. Firstly, the account logic is applicable to the majority of companies, with the exception of a select few whose data is stored in specific files. These accounts appear to adhere to the IRS's reporting nomenclature for accounts. Secondly, the consistency of data from previous years is uncertain. It would be prudent to review which files and companies align with the logic developed here and conduct an inventory of available data. Subsequent efforts could focus on deciphering the reporting logic that aligns with the IRS's nomenclature. We will focus on checking up which files adhere adequately to the SCVS's reporting nomenclature for accounts.

In [2]:
#Example of file where financial information doesn't follow the SCVS's account nomenclature
with ZipFile(f"Financials/estadosFinancieros_2022.zip","r") as zip_ref:
    zip_ref.extractall("Financials")
df = pd.read_csv("Financials/balances_2022_2.txt",sep = "\t",encoding = "latin-1")
dic = pd.read_csv("Financials/catalogo_2022_2.txt",sep = "\t",encoding = "latin-1",header = None, names = ["CÓDIGO","CUENTA"])
display(df.head(5))
display(dic.head(5))
dic[dic["CÓDIGO"]==2]
#Remove files after they've been processed
for i in [i for i in os.listdir("Financials") if "balances" in i or "catalogo" in i]:
    os.remove(f"Financials/{i}")

,AÑO,EXPEDIENTE,RUC,NOMBRE,RAMA_ACTIVIDAD,DESCRIPCION_RAMA,CIIU,CUENTA_10,CUENTA_1025,CUENTA_1030,...,CUENTA_7903,CUENTA_7904,CUENTA_7905,CUENTA_7906,CUENTA_7907,CUENTA_7908,CUENTA_7991,CUENTA_7992,CUENTA_7999,Unnamed: 932
0,2022,75761,991369643001,BIASONI S.A.,L,ACTIVIDADES INMOBILIARIAS.,L6810.01,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
1,2022,86143,1791401085001,FERTILIZANTES Y QUIMICOS FERTIQUIM S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4669.12,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
2,2022,120579,992423056001,FUMITODO S.A.,N,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,N8129.94,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
3,2022,128735,992534737001,CONTROLTELCO S.A.,L,ACTIVIDADES INMOBILIARIAS.,L6810.01,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN
4,2022,136186,992646772001,KMONPE S.A.,C,INDUSTRIAS MANUFACTURERAS.,C1512.01,0,0,0,...,0,0,0,0,0,0,0,0,0,NaN


,CÓDIGO,CUENTA
0,10,INGRESO CON PARTES RELACIONADAS EN PARAISOS FI...
1,1025,(=) Utilidad Bruta
2,1030,TOTAL GASTOS OPERACIONALES
3,1040,(=) Utilidad Operacional
4,1055,TOTAL GASTOS FINANCIEROS Y OTROS GASTOS NO OPE...


As observed in the example above, the file 'balances_2022_2' does not contain any accounts with the same numbering as those found in the initially analyzed file, 'balances_2022_1.csv.' This discrepancy arises because 'balances_2022_2' utilizes the account nomenclature of the IRS, for which we currently lack consistent examples or documentation.

In the case of the year 2022, we encountered two distinct files: one adhering to the IRS notation and the other following the SCVS convention. This dichotomy makes it relatively straightforward to identify which companies are compatible with the 'sumup' function. However, this simplicity does not extend to all years, where we may encounter up to seven different files, each containing distinct sets of companies. Furthermore, these files may adopt either the SCVS or the IRS notation without adhering to a discernible pattern or logic.

In [40]:
def open_data(year):
    #Decompress 2022's financial statements
    with ZipFile(f"Financials/estadosFinancieros_{year}.zip","r") as zip_ref:
        zip_ref.extractall("Financials")
    #Read Financial Statements
    balance_files = [i for i in os.listdir("Financials") if "balance" in i]
    catalog_files = [i for i in os.listdir("Financials") if "catalog" in i]
    #Store the balance sheets of years within a list
    balance_dfs = []
    catalog_dfs = []
    #Save balance files
    for file in balance_files:
        df = pd.read_csv(f"Financials/{file}",sep = "\t",encoding = "latin-1",low_memory = False)
        balance_dfs.append(df)
    #Save catalog files
    for file in catalog_files:
        dic = pd.read_csv(f"Financials/{file}",sep = "\t",encoding = "latin-1",header = None, names = ["CÓDIGO","CUENTA"],low_memory = False)
        catalog_dfs.append(dic)
    # Remove files once stored into RAM
    for i in [i for i in os.listdir("Financials") if "balances" in i or "catalogo" in i]:
        os.remove(f"Financials/{i}")
    #Set the correct dtypes
    for df in balance_dfs:
        for i in df.columns:
            df[i] = df[i].astype(str)
        #Set numeric variables to string.
        for i in [j for j in df.columns if "CUENTA" in j]:
            df[i] = df[i].apply(lambda x : x.replace(",","."))
            df[i] = df[i].apply(float)
    return balance_dfs, catalog_dfs

In [41]:
balance_dfs,catalog_dfs = open_data(2021)

In [ ]:
def checkup(df):
    if not df[df["CÓDIGO"] == 1].empty:
        return True
    else:
        return False

In [8]:
#Verify the number of companies that adhere to which account nomenclature for 2021
[display(i) for i in balance_dfs]

,AÑO,EXPEDIENTE,RUC,NOMBRE,RAMA_ACTIVIDAD,DESCRIPCION_RAMA,CIIU,CUENTA_1,CUENTA_101,CUENTA_10101,...,CUENTA_80004,CUENTA_80005,CUENTA_80006,CUENTA_80007,CUENTA_80008,CUENTA_80009,CUENTA_801,CUENTA_80101,CUENTA_80102,Unnamed: 629
0,2021,2,1790004724001,ACERIA DEL ECUADOR CA ADELCA.,C,INDUSTRIAS MANUFACTURERAS.,C2410.25,4.357896e+08,2.204669e+08,27231350.02,...,0.0,0.00,0.0,0.0,0.0,0.0,26989336.77,0.0,0.0,nan
1,2021,83,1890000130001,ASSA-AUTOMOTORES DE LA SIERRA S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4510.01,4.907645e+07,3.122273e+07,3912505.60,...,0.0,0.00,0.0,0.0,0.0,0.0,204739.45,0.0,0.0,nan
2,2021,85,1790014797001,AUTOMOTORES Y ANEXOS S.A. (A.Y.A.S.A.),G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4510.01,1.272657e+08,6.194282e+07,19368944.22,...,0.0,-110256.03,0.0,0.0,0.0,0.0,7500641.61,0.0,0.0,nan
3,2021,94,1790011860001,GODDARD CATERING GROUP QUITO S.A.,I,ACTIVIDADES DE ALOJAMIENTO Y DE SERVICIO DE CO...,I5610.01,3.918564e+06,1.302371e+06,225226.52,...,0.0,0.00,0.0,0.0,0.0,0.0,78484.63,0.0,0.0,nan
4,2021,168,1790009459001,CASABACA S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4510.01,7.954599e+07,5.464950e+07,10984543.95,...,0.0,0.00,0.0,0.0,0.0,0.0,2458876.02,0.0,0.0,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7992,2021,746528,491533830001,ACCG APORTE AL CAMBIO CLIMATICO GLOBAL S.A.,M,"ACTIVIDADES PROFESIONALES, CIENTÍFICAS Y TÉCNI...",M7490.23,1.000000e+03,1.000000e+03,1000.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,nan
7993,2021,746628,1793199783001,AMERICAN-RED-ASSOCIATION CIA.LTDA.,Q,ACTIVIDADES DE ATENCIÓN DE LA SALUD HUMANA Y D...,Q8690.11,4.000000e+02,4.000000e+02,400.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,nan
7994,2021,746900,1091796152001,FLORES LA CASTELLANA FLC CIA.LTDA.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4620.15,1.000000e+03,1.000000e+03,1000.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,nan
7995,2021,747911,1791786300001,AGESOTEC CIA.LTDA. AGENCIA ASESORA PRODUCTORA ...,K,ACTIVIDADES FINANCIERAS Y DE SEGUROS.,K6621.00,4.000000e+02,4.000000e+02,400.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,nan


,AÑO,EXPEDIENTE,RUC,NOMBRE,RAMA_ACTIVIDAD,DESCRIPCION_RAMA,CIIU,CUENTA_10,CUENTA_11,CUENTA_12,...,CUENTA_919,CUENTA_920,CUENTA_925,CUENTA_94,CUENTA_950,CUENTA_960,CUENTA_97,CUENTA_98,CUENTA_999,Unnamed: 875
0,2021,320915,1793194272001,ARTOPIN CONSULTORÍA & CONSTRUCCIÓN S.A.S.,F,CONSTRUCCIÓN.,F4100.10,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nan


,AÑO,EXPEDIENTE,RUC,NOMBRE,RAMA_ACTIVIDAD,DESCRIPCION_RAMA,CIIU,CUENTA_10,CUENTA_1025,CUENTA_1030,...,CUENTA_7903,CUENTA_7904,CUENTA_7905,CUENTA_7906,CUENTA_7907,CUENTA_7908,CUENTA_7991,CUENTA_7992,CUENTA_7999,Unnamed: 932
0,2021,1,1790013731001,ACEITES TROPICALES SOCIEDAD ANONIMA ATSA,A,"AGRICULTURA, GANADERÍA, SILVICULTURA Y PESCA.",A0126.01,0.0,-144370.63,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,144370.63,32319.42,176690.05,nan
1,2021,3,1790008959001,ACERO COMERCIAL ECUATORIANO S.A.,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4610.03,0.0,-7820189.20,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,7820189.17,3745372.96,11565562.10,nan
2,2021,11,1790044149001,AEROVIAS DEL CONTINENTE AMERICANO S.A. AVIANCA,H,TRANSPORTE Y ALMACENAMIENTO.,H5110.01,0.0,2580112.74,1060842.44,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,2989873.19,1062455.64,4052328.83,nan
3,2021,22,1790023516001,AGENCIAS Y REPRESENTACIONES CORDOVEZ SA,G,COMERCIO AL POR MAYOR Y AL POR MENOR REPARACIÓ...,G4630.95,0.0,2770786.43,2334688.45,...,0.0,1.790835e+12,0.0,0.0,0.0,0.0,23925543.90,2732988.12,26658532.00,nan
4,2021,40,1790340155001,AGUA Y GAS DE SILLUNCHI SA,C,INDUSTRIAS MANUFACTURERAS.,C2011.11,0.0,-44667.89,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,818280.42,0.00,818280.42,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92484,2021,744522,1793196787001,AGADANUTRICOM CIA.LTDA.,Q,ACTIVIDADES DE ATENCIÓN DE LA SALUD HUMANA Y D...,Q8690.11,0.0,0.00,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,nan
92485,2021,745084,1793197647001,COMPAÑIA DE SEGURIDAD FALCÓN RIVERA TOBAR FARI...,N,ACTIVIDADES DE SERVICIOS ADMINISTRATIVOS Y DE ...,N8010.03,0.0,0.00,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,nan
92486,2021,745821,2390627745001,WORK & SOLUTIONS MELCING S.A.,D,"SUMINISTRO DE ELECTRICIDAD, GAS, VAPOR Y AIRE ...",D3510.02,0.0,0.00,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,nan
92487,2021,746472,791841130001,COMPAÑIA DE TRANSPORTE EN TAXI TRANS RIOBONITO...,H,TRANSPORTE Y ALMACENAMIENTO.,H4922.02,0.0,0.00,0.00,...,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.00,0.00,0.00,nan


[None, None, None]

In [6]:
a = catalog_dfs[2]
b = a["CUENTA"].values

,CÓDIGO,CUENTA
11,1102,COSTOS Y GASTOS PARA GENERAR INGRESOS SUJETOS ...


In [53]:
c = catalog_dfs[2]
c[c["CUENTA"].str.contains("RELACIONADA")]

,CÓDIGO,CUENTA
0,10,INGRESO CON PARTES RELACIONADAS EN PARAISOS FI...
8,11,EGRESO CON PARTES RELACIONADAS EN PARAISOS FIS...
60,13,ACTIVO CON PARTES RELACIONADAS EN OTROS REGIMENES
61,14,PASIVO CON PARTES RELACIONADAS EN OTROS REGIMENES
62,15,INGRESO CON PARTES RELACIONADAS EN OTROS REGIM...
...,...,...
884,7279,VALOR NO DEDUCIBLE GASTOS INTERESES CON INSTIT...
885,7281,GASTO INTERESES CON INSTITUCIONES FINANCIERAS ...
886,7282,VALOR NO DEDUCIBLE INTERESES CON INSTITUCIONES...
887,7284,GASTO INTERESES CON INSTITUCIONES FINANCIERAS ...


In [2]:
#Identify parent accounts 
tier_1 = ["TOTAL DEL ACTIVO","TOTAL DEL PASIVO","TOTAL DEL PATRIMONIO"]
tier_2 = ["TOTAL ACTIVOS CORRIENTES","TOTAL ACTIVOS NO CORRIENTES","TOTAL PASIVOS CORRIENTES","TOTAL PASIVOS NO CORRIENTES","RESERVAS","RESULTADOS ACUMULADOS","OTROS RESULTADOS INTEGRALES ACUMULADOS"]
tier_3 = ["CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES", "OTROS ACTIVOS FINANCIEROS CORRIENTES","PORCIÓN CORRIENTE DE ARRENDAMIENTOS FINANCIEROS POR COBRAR","IMPORTE BRUTO ADEUDADO POR LOS CLIENTES POR EL TRABAJO EJECUTADO EN CONTRATOS DE CONSTRUCCIÓN","ACTIVOS POR IMPUESTOS CORRIENTES","INVENTARIOS","ACTIVOS NO CORRIENTES MANTENIDOS PARA LA VENTA","ACTIVOS BIOLÓGICOS","GASTOS PAGADOS POR ANTICIPADO (PREPAGADOS)","PROPIEDADES, PLANTA Y EQUIPO","ACTIVOS INTANGIBLES","PROPIEDADES DE INVERSIÓN","ACTIVOS BIOLÓGICOS","ACTIVOS PARA EXPLORACIÓN, EVALUACIÓN Y EXPLOTACIÓN DE RECURSOS MINERALES","INVERSIONES NO CORRIENTES","CUENTAS Y DOCUMENTOS POR COBRAR NO CORRIENTES","OTROS ACTIVOS FINANCIEROS NO CORRIENTES","PORCIÓN NO CORRIENTE DE ARRENDAMIENTOS FINANCIEROS POR COBRAR","ACTIVOS POR IMPUESTOS DIFERIDOS","CUENTAS Y DOCUMENTOS POR PAGAR CORRIENTES","OBLIGACIONES CON INSTITUCIONES FINANCIERAS - CORRIENTES","OTROS PASIVOS FINANCIEROS","PASIVOS CORRIENTES POR BENEFICIOS A LOS EMPLEADOS","PROVISIONES CORRIENTES","PASIVOS POR INGRESOS DIFERIDOS","OTROS PASIVOS CORRIENTES","CUENTAS Y DOCUMENTOS POR PAGAR NO CORRIENTES","OBLIGACIONES CON INSTITUCIONES FINANCIERAS - NO CORRIENTES","OTROS PASIVOS FINANCIEROS NO CORRIENTES","PASIVOS NO CORRIENTES POR BENEFICIOS A LOS EMPLEADOS","PROVISIONES NO CORRIENTES","PASIVOS POR INGRESOS DIFERIDOS","OTROS PASIVOS NO CORRIENTES","SUPERÁVIT DE REVALUACIÓN ACUMULADO"]
tier_4 = ["CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES CORRIENTES","OTRAS CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES","PLANTAS VIVAS Y FRUTOS EN CRECIMIENTO","ANIMALES VIVOS","TERRENOS","EDIFICIOS Y OTROS INMUEBLES (EXCEPTO TERRENOS)","NAVES, AERONAVES, BARCAZAS Y SIMILARES","MAQUINARIA, EQUIPO, INSTALACIONES Y ADECUACIONES","PLANTAS PRODUCTORAS (AGRICULTURA)","PROPIEDADES, PLANTA Y EQUIPO POR CONTRATOS DE ARRENDAMIENTO FINANCIERO","(-) DEPRECIACIÒN ACUMULADA DE PROPIEDADES, PLANTA Y EQUIPO","TERRENOS","EDIFICIOS","PLANTAS VIVAS Y FRUTOS EN CRECIMIENTO","ANIMALES VIVOS","EN SUBSIDIARIAS","EN ASOCIADAS","EN NEGOCIOS CONJUNTOS","CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES NO CORRIENTES","OTRAS CUENTAS Y DOCUMENTOS POR COBRAR NO CORRIENTES","POR CRÉDITOS FISCALES NO UTILIZADOS","CUENTAS Y DOCUMENTOS POR PAGAR COMERCIALES CORRIENTES","OTRAS CUENTAS Y DOCUMENTOS POR PAGAR CORRIENTES","RELACIONADAS","NO RELACIONADAS","CUENTAS Y DOCUMENTOS POR PAGAR COMERCIALES NO CORRIENTES","OTRAS CUENTAS Y DOCUMENTOS POR PAGAR NO CORRIENTES","RELACIONADAS","NO RELACIONADAS"]
tier_5 = ["RELACIONADAS","NO RELACIONADAS","A ACCIONISTAS, SOCIOS, PARTICIPES, BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL","DIVIDENDOS POR COBRAR","OTRAS RELACIONADAS","OTRAS NO RELACIONADAS","RELACIONADAS","NO RELACIONADAS","A ACCIONISTAS. SOCIOS, PARTICIPES. BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL","OTRAS RELACIONADAS","OTRAS NO RELACIONADAS","RELACIONADAS","NO RELACIONADAS","A ACCIONISTAS. SOCIOS, PARTÍCIPES. BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL","DIVIDENDOS POR PAGAR","OTRAS RELACIONADAS","OTRAS NO RELACIONADAS","RELACIONADAS","NO RELACIONADAS","A ACCIONISTAS, SOCIOS, PARTÍCIPES. BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL","OTRAS RELACIONADAS","OTRAS NO RELACIONADAS"]

In [43]:
tier_list = [tier_1,tier_2,tier_3,tier_4,tier_5]
def build_tree(tier_list):
    tree = {}
    #Initialize child tier
    tier = len(tier_list)-1
    #Identify parents and children
    while tier > 0:
        print((tier-1,tier))
        #Select groups of parents and children, based on child tier.
        parents = tier_list[tier-1]
        children = tier_list[tier]
        for parent in parents:
            if tier == len(tier_list)-1:
                #Ask user to declare number of children in parent account
                num = int(input(f"Lowest Tier Parent: {parent}. Declare number of children in account"))
                #Slice of children accounts corresponding to specific parent
                chs = children[:num]
                children = [child for child in children if child not in chs]
                tree[parent] = chs
                
            else:
                num = int(input(f"High Tier Parent: {parent}. Declare number of children in account"))
                chs = children[:num]
                children = [child for child in children if child not in chs]
                slice = dict([(key,tree.pop(key)) for key in chs])
                tree[parent] = slice
        #Reduce the parent tier by 1
        tier -= 1
    return tree

In [33]:
#Build the tree
tree = build_tree(tier_list)
display(tree)
#Save the tree in a json file
import json
with open("Trees/bs_account_tree_2021.json","w",encoding = "utf-8") as f:
    json.dump(tree,f,indent = 5,ensure_ascii = False)

(3, 4)


{'CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES CORRIENTES': ['RELACIONADAS',
  'NO RELACIONADAS'],
 'OTRAS CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES': ['A ACCIONISTAS, SOCIOS, PARTICIPES, BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL',
  'DIVIDENDOS POR COBRAR',
  'OTRAS RELACIONADAS',
  'OTRAS NO RELACIONADAS'],
 'PLANTAS VIVAS Y FRUTOS EN CRECIMIENTO': [],
 'ANIMALES VIVOS': [],
 'TERRENOS': [],
 'EDIFICIOS Y OTROS INMUEBLES (EXCEPTO TERRENOS)': [],
 'NAVES, AERONAVES, BARCAZAS Y SIMILARES': [],
 'MAQUINARIA, EQUIPO, INSTALACIONES Y ADECUACIONES': [],
 'PLANTAS PRODUCTORAS (AGRICULTURA)': [],
 'PROPIEDADES, PLANTA Y EQUIPO POR CONTRATOS DE ARRENDAMIENTO FINANCIERO': [],
 '(-) DEPRECIACIÒN ACUMULADA DE PROPIEDADES, PLANTA Y EQUIPO': [],
 'EDIFICIOS': [],
 'EN SUBSIDIARIAS': [],
 'EN ASOCIADAS': [],
 'EN NEGOCIOS CONJUNTOS': [],
 'CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES NO CORRIENTES': ['A ACCIONISTAS. SOCIOS, PARTICIPES. BENEFICIARIOS U OTROS TITULARES DE 

(2, 3)
SECOND TIER


['CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES CORRIENTES',
 'OTRAS CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES',
 'PLANTAS VIVAS Y FRUTOS EN CRECIMIENTO',
 'ANIMALES VIVOS',
 'TERRENOS',
 'EDIFICIOS Y OTROS INMUEBLES (EXCEPTO TERRENOS)',
 'NAVES, AERONAVES, BARCAZAS Y SIMILARES',
 'MAQUINARIA, EQUIPO, INSTALACIONES Y ADECUACIONES',
 'PLANTAS PRODUCTORAS (AGRICULTURA)',
 'PROPIEDADES, PLANTA Y EQUIPO POR CONTRATOS DE ARRENDAMIENTO FINANCIERO',
 '(-) DEPRECIACIÒN ACUMULADA DE PROPIEDADES, PLANTA Y EQUIPO',
 'TERRENOS',
 'EDIFICIOS',
 'PLANTAS VIVAS Y FRUTOS EN CRECIMIENTO',
 'ANIMALES VIVOS',
 'EN SUBSIDIARIAS',
 'EN ASOCIADAS',
 'EN NEGOCIOS CONJUNTOS',
 'CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES NO CORRIENTES',
 'OTRAS CUENTAS Y DOCUMENTOS POR COBRAR NO CORRIENTES',
 'POR CRÉDITOS FISCALES NO UTILIZADOS',
 'CUENTAS Y DOCUMENTOS POR PAGAR COMERCIALES CORRIENTES',
 'OTRAS CUENTAS Y DOCUMENTOS POR PAGAR CORRIENTES',
 'RELACIONADAS',
 'NO RELACIONADAS',
 'CUENTAS Y DOCUMENTOS POR PAGAR CO

Parent CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES
SECOND TIER


...

['TOTAL PASIVOS CORRIENTES',
 'TOTAL PASIVOS NO CORRIENTES',
 'RESERVAS',
 'RESULTADOS ACUMULADOS',
 'OTROS RESULTADOS INTEGRALES ACUMULADOS']

Parent TOTAL DEL PASIVO
SECOND TIER


['RESERVAS', 'RESULTADOS ACUMULADOS', 'OTROS RESULTADOS INTEGRALES ACUMULADOS']

Parent TOTAL DEL PATRIMONIO


{'TOTAL DEL ACTIVO': {'TOTAL ACTIVOS CORRIENTES': {'CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES': {'CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES CORRIENTES': ['RELACIONADAS',
     'NO RELACIONADAS'],
    'OTRAS CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES': ['A ACCIONISTAS, SOCIOS, PARTICIPES, BENEFICIARIOS U OTROS TITULARES DE DERECHOS REPRESENTATIVOS DE CAPITAL',
     'DIVIDENDOS POR COBRAR',
     'OTRAS RELACIONADAS',
     'OTRAS NO RELACIONADAS']},
   'OTROS ACTIVOS FINANCIEROS CORRIENTES': {},
   'PORCIÓN CORRIENTE DE ARRENDAMIENTOS FINANCIEROS POR COBRAR': {},
   'IMPORTE BRUTO ADEUDADO POR LOS CLIENTES POR EL TRABAJO EJECUTADO EN CONTRATOS DE CONSTRUCCIÓN': {},
   'ACTIVOS POR IMPUESTOS CORRIENTES': {},
   'INVENTARIOS': {},
   'ACTIVOS NO CORRIENTES MANTENIDOS PARA LA VENTA': {},
   'ACTIVOS BIOLÓGICOS': {'EN ASOCIADAS': [], 'EN NEGOCIOS CONJUNTOS': []},
   'GASTOS PAGADOS POR ANTICIPADO (PREPAGADOS)': {}},
  'TOTAL ACTIVOS NO CORRIENTES': {'PROPIEDADES, PLANTA Y EQUIPO': {'TERRENO

The method above helps me build account trees more efficiently than manually making a dictionary with the parent accounts.

The next step is to add the values for the leaf nodes inside the tree. Once that is done, we can focus on 

In [39]:
a = {"A":[1,2,3],
     "B":[4,5,6],
     "C":[7,8,9,10]}
display(a)

parent_list = ["a","b"]
child_list = ["A","B","C"]
for parent in parent_list:
    print(child_list)
    l_in = int(input(f"parent is {parent}, provide number of children in account"))
    childs = child_list[:l_in]
    child_list = [i for i in child_list if i not in childs]
    slice = dict([(key,a.pop(key)) for key in childs])
    display(slice)
    a[f"key_{parent}"] = slice
    display(f"original dict turns to: {a}")

{'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9, 10]}

['A', 'B', 'C']


{'A': [1, 2, 3], 'B': [4, 5, 6]}

"original dict turns to: {'C': [7, 8, 9, 10], 'key_a': {'A': [1, 2, 3], 'B': [4, 5, 6]}}"

['C']


{'C': [7, 8, 9, 10]}

"original dict turns to: {'key_a': {'A': [1, 2, 3], 'B': [4, 5, 6]}, 'key_b': {'C': [7, 8, 9, 10]}}"

In [32]:
table[0].iloc[25:]

,Unnamed: 0,FORMULARIO 101,DECLARACIÓN DEL IMPUESTO A LA RENTA Y PRESENTACIÓN DE BALANCES FORMULARIO ÚNICO SOCIEDADES Y ESTABLECIMIENTOS PERMANENTES,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
25,ACTIVO,NaN,NaN,NaN,NaN,NaN,NaN
26,ACTIVOS CORRIENTES,NaN,NaN,NaN,NaN,NaN,NaN
27,NaN,Efectivo y equivalentes al efectivo,NaN,NaN,311.0,NaN,281673.22
28,NaN,CUENTAS Y DOCUMENTOS POR COBRAR CORRIENTES,NaN,NaN,NaN,NaN,NaN
29,NaN,NaN,CUENTAS Y DOCUMENTOS POR COBRAR COMERCIALES CO...,NaN,NaN,NaN,NaN
30,NaN,NaN,RELACIONADAS,NaN,NaN,NaN,NaN
31,NaN,NaN,Locales,NaN,312.0,NaN,104826.14
32,NaN,NaN,Del exterior,NaN,313.0,NaN,0.00
33,NaN,NaN,(-) Deterioro acumulado del valor de cuentas y...,NaN,314.0,NaN,0.00
34,NaN,NaN,NO RELACIONADAS,NaN,NaN,NaN,NaN
